#### 입력 : ./make_file/division/first_round/서울시_자동차도로_분할_{숫자}.csv 
#### 출력 : ./make_file/division/second_round/서울시_도로_{숫자}.csv

## 서울시 각 동별로 도로 수가 몇 개 인지 파악하기 위한 것

In [1]:
import pandas as pd 
import requests
from tqdm import tqdm 
import os

In [2]:
import pandas as pd 
import requests
from tqdm import tqdm 
import os
import geopandas as gpd 
from shapely.geometry import Point, Polygon, MultiPolygon 
import geopandas as gpd
import pyproj
from shapely.ops import transform

In [3]:
# 파일이 저장된 폴더 경로
folder_path = "./make_file/division/first_round/" 
file_encoding = "UTF-8"

output_file_url = "./make_file/division/second_round/서울시_도로"

In [4]:
dong_url = "../데이터_보정/data/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"
dong_encoding = "CP949"

In [5]:
# 행정동 경계 데이터 로드
gdf = gpd.read_file(dong_url, encoding=dong_encoding)

In [6]:
# 좌표 변환 함수 정의
project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:5186", always_xy=True).transform

In [7]:
# 조건을 만족하는 파일 목록
valid_files = [] 

In [8]:
for temp_file_name in os.listdir(folder_path):
    file_name = temp_file_name.replace("_", " ") 
    if "서울시 자동차도로 분할" in file_name and file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, temp_file_name)  # 안전한 경로 조합

        try:
            # CSV 파일 전체 읽기
            df = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

            # 컬럼명 앞뒤 공백 제거
            df.columns = df.columns.str.strip()

            # 필요한 컬럼이 존재하는지 확인
            if {"중앙점_위도", "중앙점_경도"}.issubset(df.columns):
                check = file_name.replace("서울시 자동차도로 분할 " , "") 
                temp_check_file_url = output_file_url + "_" + check
                
                output_check = os.path.isfile(temp_check_file_url) 
                
                if output_check is False :
                    valid_files.append(file_path)
                else :
                    print(f"{temp_check_file_url} 파일이 이미 있습니다.")
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")


./make_file/division/second_round/서울시_도로_17.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_03.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_02.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_16.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_14.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_15.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_01.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_05.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_11.csv 파일이 이미 있습니다.
./make_file/division/second_round/서울시_도로_10.csv 파일이 이미 있습니다.


In [9]:
len(valid_files)

7

In [10]:
def get_admin_dong_name(lon, lat):
    # 위경도 → EPSG:5186 좌표로 변환
    point = transform(project, Point(lon, lat))
    # print("변환된 좌표:", point)

    for _, row in gdf.iterrows():
        geom = row['geometry']
        if isinstance(geom, Polygon):
            if geom.contains(point):
                return row['ADM_NM']
        elif isinstance(geom, MultiPolygon):
            for poly in geom.geoms:
                if poly.contains(point):
                    return row['ADM_NM']
    return None


In [12]:
for file_path in valid_files:  
    data = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

    lat_list = data["중앙점_위도"].to_list()
    lon_list = data["중앙점_경도"].to_list() 

    is_none = 0
    
    dong_list = [] 
    print(f"작업 중 : {file_path}")
    for i in tqdm(range(len(lat_list))):
        lat = lat_list[i]
        lon = lon_list[i]
        dong = get_admin_dong_name(lon, lat)
        dong_list.append(dong)   
        if dong is None :
            is_none += 1
            
    print(f"불러온 행정동의 수 : {len(dong_list)}")
    print(f"None의 수 : {is_none}")

    data["행정동"] = dong_list
    
    try :
        temp_out_url = file_path.replace("./make_file/division/first_round/서울시_자동차도로_분할_" , "") 
        save_file_url = output_file_url + "_" + temp_out_url 
        data.to_csv(save_file_url, index=False)
        print(f"{save_file_url} 파일 저장 완료.", end ="\n\n")
    except OSError as e :
        print(e)

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_04.csv


100%|█████████████████████████████████████| 10000/10000 [08:46<00:00, 19.00it/s]


불러온 행정동의 수 : 10000
None의 수 : 0
./make_file/division/second_round/서울시_도로_04.csv 파일 저장 완료.

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_12.csv


100%|█████████████████████████████████████| 10000/10000 [08:47<00:00, 18.94it/s]


불러온 행정동의 수 : 10000
None의 수 : 0
./make_file/division/second_round/서울시_도로_12.csv 파일 저장 완료.

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_06.csv


...

불러온 행정동의 수 : 10000
None의 수 : 0
./make_file/division/second_round/서울시_도로_13.csv 파일 저장 완료.

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_09.csv


100%|█████████████████████████████████████| 10000/10000 [08:06<00:00, 20.57it/s]


불러온 행정동의 수 : 10000
None의 수 : 0
./make_file/division/second_round/서울시_도로_09.csv 파일 저장 완료.

작업 중 : ./make_file/division/first_round/서울시_자동차도로_분할_08.csv


100%|█████████████████████████████████████| 10000/10000 [09:03<00:00, 18.39it/s]

불러온 행정동의 수 : 10000
None의 수 : 0
./make_file/division/second_round/서울시_도로_08.csv 파일 저장 완료.

